In [1]:
import os
import re
import pandas as pd
import numpy as np
import pickle
import joblib
from sqlalchemy import create_engine
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, classification_report
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import xgboost as xgb

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Remote end closed connection without response


False

In [ ]:
conn = create_engine('sqlite:///data/disaster_response.db')
df = pd.read_sql_table('df', conn)

In [ ]:
# pd.read_sql_query('SELECT * FROM df;', conn)

In [ ]:
df.head(1)

In [ ]:
# X = df['message'][:5]
# y = df.drop(['id', 'message', 'original', 'genre_news', 'genre_social', 'genre'], axis = 1).iloc[:5,]

In [ ]:
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre_news', 'genre_social', 'genre'], axis = 1)

In [48]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'url_placeholder')
        
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
        
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    stopwords_ = stopwords.words("english")
    words = [word for word in tokens if word not in stopwords_]
    
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_token)
        
    return clean_tokens

def build_model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([
                    ('text_pipeline', Pipeline([
                                ('vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer())
                            ])),
                    ('starting_verb', StartingVerbExtractor())
                        ])),
        ('clf', MultiOutputClassifier(xgb.XGBClassifier()))
                    ])
    return pipeline

class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            
            try:
                first_word, first_tag = pos_tags[0]
            except IndexError:
                pass
            else:
                if first_tag in ['VB', 'VBP']:
                    return True
        return False
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)
            

def eval_metrics(actual, predicted, column_names):
    """
    calculate evaluation metrics for model
    
    INPUT: 
    --actual : array of actual labels
    --predicted: array of predicted labels
    
    OUTPUT:
    --- Pandas Dataframe containing the accuracy, precision, recall 
        and f1 score for a given set of actual and predicted labels.
    """
    metrics = {'accuracy':[], 'precision':[], 'recall':[], 'f1':[]}
    for i in range(len(column_names)):
        accuracy = accuracy_score(actual.iloc[:,i], predicted[:,i])
        precision = precision_score(actual.iloc[:,i], predicted[:,i], average='weighted')
        recall = recall_score(actual.iloc[:,i], predicted[:,i], average='weighted')
        f1 = f1_score(actual.iloc[:,i], predicted[:,i], average='weighted', )
        
        metrics['accuracy'].append(accuracy)
        metrics['precision'].append(precision)
        metrics['recall'].append(recall)
        metrics['f1'].append(f1)
        print(classification_report(actual.iloc[:,i], predicted[:,i]))
        
    df = pd.DataFrame(metrics, index=column_names)
    return df

def gridsCV_scorer(actual, predicted):
    """
    calculate median median f1 score for all output classifiers
    INPUT: 
    --actual : array of actual labels
    --predicted: array of predicted labels
    
    OUTPUT: Median Output Score
    """
    f1 = []
    for i in range(actual.shape[1]):
        f1_scr = f1_score(actual.iloc[:,i], predicted[:,i], average='weighted')
        f1.append(f1_scr)
        return np.median(f1)

In [49]:
%%time
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=19)
pipeline = build_model_pipeline()
pipeline.fit(X_train, y_train)

[('unicef', 'JJ'), ('ha', 'NN'), ('offered', 'VBD'), ('20', 'CD'), ('squatting', 'VBG'), ('pan', 'NN'), ('to', 'TO'), ('the', 'DT'), ('mosque', 'NN'), ('and', 'CC'), ('a', 'DT'), ('temporary', 'JJ'), ('building', 'NN'), ('is', 'VBZ'), ('being', 'VBG'), ('constructed', 'VBN'), ('to', 'TO'), ('supplement', 'VB'), ('the', 'DT'), ('existing', 'VBG'), ('shelter', 'NN'), ('at', 'IN'), ('the', 'DT'), ('mosque', 'NN'), ('with', 'IN'), ('four', 'CD'), ('semi', 'NN'), ('permanent', 'JJ'), ('toilet', 'NN'), ('and', 'CC'), ('a', 'DT'), ('bathing', 'NN'), ('site', 'NN')]
[('what', 'WDT'), ('are', 'VBP'), ('the', 'DT'), ('authority', 'NN'), ('saying', 'VBG'), ('about', 'IN'), ('the', 'DT'), ('functioning', 'NN'), ('of', 'IN'), ('the', 'DT'), ('school', 'NN'), ('in', 'IN'), ('the', 'DT'), ('rest', 'NN'), ('of', 'IN'), ('the', 'DT'), ('country', 'NN')]
[('i', 'NNS'), ('have', 'VBP'), ('a', 'DT'), ('lot', 'NN'), ('of', 'IN'), ('problem', 'NN'), ('and', 'CC'), ('i', 'NN'), ('do', 'VBP'), ('not', 'RB'), 

[('he', 'PRP'), ('further', 'RB'), ('hoped', 'VBD'), ('that', 'IN'), ('the', 'DT'), ('recent', 'JJ'), ('increased', 'VBN'), ('participation', 'NN'), ('by', 'IN'), ('the', 'DT'), ('united', 'JJ'), ('state', 'NN'), ('in', 'IN'), ('climate', 'NN'), ('debate', 'NN'), ('indicated', 'VBD'), ('it', 'PRP'), ('intention', 'NN'), ('to', 'TO'), ('play', 'VB'), ('a', 'DT'), ('major', 'JJ'), ('role', 'NN')]
[('we', 'PRP'), ('need', 'VBP'), ('help', 'VB'), ('in', 'IN'), ('croix', 'NN'), ('de', 'FW'), ('bouquet', 'FW'), ('call', 'VB'), ('me', 'PRP'), ('for', 'IN'), ('more', 'JJR'), ('info', 'NN')]
[('poliomyelitis', 'NN'), ('according', 'VBG'), ('to', 'TO'), ('the', 'DT'), ('world', 'NN'), ('initiative', 'NN'), ('for', 'IN'), ('the', 'DT'), ('eradication', 'NN'), ('of', 'IN'), ('polio4', 'NN'), ('there', 'EX'), ('is', 'VBZ'), ('a', 'DT'), ('potential', 'JJ'), ('threat', 'NN'), ('of', 'IN'), ('a', 'DT'), ('polio', 'JJ'), ('epidemic', 'NN'), ('in', 'IN'), ('west', 'NN'), ('africa', 'NN')]
[('note', 'NN

[('and', 'CC'), ('the', 'DT'), ('overall', 'JJ'), ('number', 'NN'), ('of', 'IN'), ('death', 'NN'), ('attributed', 'VBN'), ('to', 'TO'), ('pro', 'VB'), ('government', 'NN'), ('force', 'NN'), ('which', 'WDT'), ('include', 'VBP'), ('the', 'DT'), ('ana', 'NN'), ('afghan', 'JJ'), ('national', 'JJ'), ('army', 'NN'), ('anp', 'NN'), ('afghan', 'IN'), ('national', 'JJ'), ('police', 'NNS'), ('nd', 'JJ'), ('national', 'JJ'), ('directorate', 'NN'), ('of', 'IN'), ('security', 'NN'), ('and', 'CC'), ('international', 'JJ'), ('military', 'JJ'), ('force', 'NN'), ('marginally', 'RB'), ('exceeds', 'VBZ'), ('that', 'IN'), ('caused', 'VBN'), ('by', 'IN'), ('antigovernment', 'JJ'), ('force', 'NN')]
[('would', 'MD'), ('like', 'VB'), ('to', 'TO'), ('know', 'VB'), ('if', 'IN'), ('school', 'NN'), ('will', 'MD'), ('resume', 'VB'), ('again', 'RB'), ('or', 'CC'), ('not', 'RB')]
[('help', 'VB'), ('me', 'PRP'), ('because', 'IN'), ('i', 'NN'), ('am', 'VBP'), ('alone', 'RB'), ('during', 'IN'), ('the', 'DT'), ('earthqu

[('can', 'MD'), ('you', 'PRP'), ('help', 'VB'), ('me', 'PRP')]
[('good', 'JJ'), ('evening', 'NN'), ('when', 'WRB'), ('you', 'PRP'), ('active', 'JJ'), ('50', 'CD'), ('g', 'NN'), ('sm', 'NN'), ('is', 'VBZ'), ('it', 'PRP'), ('for', 'IN'), ('international', 'JJ'), ('thank', 'NN'), ('you', 'PRP')]
[('the', 'DT'), ('route', 'JJ'), ('north', 'NN'), ('and', 'CC'), ('south', 'NN'), ('can', 'MD'), ('handle', 'VB'), ('two', 'CD'), ('lane', 'NN'), ('traffic', 'NN'), ('on', 'IN'), ('most', 'JJS'), ('of', 'IN'), ('the', 'DT'), ('coastal', 'JJ'), ('road', 'NN'), ('from', 'IN'), ('e', '$'), ('04', 'CD'), ('b043', 'NN'), ('53', 'CD'), ('8', 'CD'), ('n', 'JJ'), ('95', 'CD'), ('b030', 'NN'), ('27', 'CD'), ('4', 'CD'), ('to', 'TO'), ('e', 'VB'), ('04', 'CD'), ('b032', 'NN'), ('32', 'CD'), ('4', 'CD'), ('n', 'JJ'), ('95', 'CD'), ('b043', 'NN'), ('02', 'CD'), ('0', 'CD'), ('with', 'IN'), ('area', 'NN'), ('that', 'WDT'), ('are', 'VBP'), ('reduced', 'VBN'), ('to', 'TO'), ('single', 'JJ'), ('lane', 'NN'), ('tr

[('no', 'DT'), ('gps', 'NN'), ('coordinate', 'NN'), ('available', 'JJ'), ('from', 'IN'), ('here', 'RB')]
[('rt', 'NN'), ('cnn', 'NN'), ('find', 'VB'), ('out', 'RP'), ('how', 'WRB'), ('you', 'PRP'), ('can', 'MD'), ('help', 'VB'), ('make', 'VB'), ('a', 'DT'), ('difference', 'NN'), ('to', 'TO'), ('haiti', 'VB'), ('u2019s', 'JJ'), ('earthquake', 'NN'), ('victim', 'NN'), ('please', 'NN'), ('visit', 'NN'), ('cnn', 'NN'), ('com', 'NN'), ('impact', 'NN')]
[('the', 'DT'), ('team', 'NN'), ('previously', 'RB'), ('tackled', 'VBD'), ('two', 'CD'), ('other', 'JJ'), ('cholera', 'NN'), ('epidemic', 'NN'), ('in', 'IN'), ('kamanyola', 'NN'), ('and', 'CC'), ('uvira', 'VB'), ('a', 'DT'), ('malaria', 'NN'), ('emergency', 'NN'), ('in', 'IN'), ('lulingo', 'NN'), ('and', 'CC'), ('two', 'CD'), ('measles', 'NNS'), ('emergency', 'NN'), ('one', 'CD'), ('in', 'IN'), ('kitutu', 'NN'), ('and', 'CC'), ('another', 'DT'), ('in', 'IN'), ('bunyakiri', 'NN'), ('which', 'WDT'), ('included', 'VBD'), ('a', 'DT'), ('vaccinati

...

[('hello', 'NN')]
[('i', 'JJ'), ('live', 'VBP'), ('in', 'IN'), ('leogane', 'NN')]
[('my', 'PRP$'), ('house', 'NN'), ('is', 'VBZ'), ('destroyed', 'VBN'), ('and', 'CC'), ('my', 'PRP$'), ('family', 'NN'), ('and', 'CC'), ('i', 'NN'), ('are', 'VBP'), ('sleeping', 'VBG'), ('in', 'IN'), ('the', 'DT'), ('street', 'NN')]
[('we', 'PRP'), ('are', 'VBP'), ('hungry', 'JJ'), ('and', 'CC'), ('haven', 'JJ'), ('t', 'NN'), ('found', 'VBD'), ('food', 'NN'), ('to', 'TO'), ('eat', 'VB')]
[('help', 'VB'), ('me', 'PRP'), ('please', 'VB')]
[('this', 'DT'), ('week', 'NN'), ('the', 'DT'), ('government', 'NN'), ('launched', 'VBD'), ('a', 'DT'), ('100', 'CD'), ('million', 'CD'), ('dollar', 'NN'), ('climate', 'NN'), ('resilience', 'NN'), ('programme', 'NN'), ('that', 'WDT'), ('will', 'MD'), ('spend', 'VB'), ('the', 'DT'), ('bulk', 'NN'), ('of', 'IN'), ('it', 'PRP'), ('fund', 'VBP'), ('around', 'RB'), ('90', 'CD'), ('million', 'CD'), ('dollar', 'NN'), ('on', 'IN'), ('infrastructure', 'NN'), ('upgrade', 'NN')]
[('st

[('the', 'DT'), ('wounded', 'VBN'), ('in', 'IN'), ('the', 'DT'), ('saint', 'NN'), ('michely', 'RB'), ('hospital', 'JJ'), ('have', 'VBP'), ('no', 'DT'), ('medicine', 'NN')]
[('this', 'DT'), ('look', 'NN'), ('like', 'IN'), ('part', 'NN'), ('of', 'IN'), ('a', 'DT'), ('message', 'NN'), ('that', 'WDT'), ('got', 'VBD'), ('cut', 'VBN'), ('off', 'RP')]
[('in', 'IN'), ('several', 'JJ'), ('country', 'NN'), ('these', 'DT'), ('national', 'JJ'), ('forest', 'NN'), ('programme', 'NN'), ('have', 'VBP'), ('been', 'VBN'), ('implemented', 'VBN'), ('with', 'IN'), ('the', 'DT'), ('support', 'NN'), ('of', 'IN'), ('the', 'DT'), ('international', 'JJ'), ('donor', 'NN'), ('community', 'NN'), ('undp', 'JJ'), ('global', 'JJ'), ('environment', 'NN'), ('facility', 'NN'), ('gef', 'NN'), ('mobilizing', 'VBG'), ('financial', 'JJ'), ('national', 'JJ'), ('and', 'CC'), ('external', 'JJ'), ('resource', 'NN'), ('for', 'IN'), ('programme', 'NN'), ('and', 'CC'), ('project', 'NN'), ('relevant', 'NN'), ('to', 'TO'), ('the', '

[('rickywussup', 'JJ'), ('fuckit', 'NN'), ('who', 'WP'), ('tryna', 'VBZ'), ('party', 'NN'), ('tonight', 'NN'), ('sippin', 'NN'), ('on', 'IN'), ('hurricane', 'NN'), ('through', 'IN'), ('out', 'IN'), ('the', 'DT'), ('hurricane', 'NN')]
[('rt', 'NN'), ('protagonistas', 'NNS'), ('impresionante', 'VBP'), ('fotograf', 'JJ'), ('u00eda', 'JJ'), ('del', 'NN'), ('palacio', 'NN'), ('presidencial', 'JJ'), ('en', 'NN'), ('hait', 'NN'), ('u00ed', 'JJ'), ('ante', 'NN'), ('y', 'NN'), ('despu', 'NN'), ('u00e9s', 'JJ'), ('http', 'NN'), ('twitpic', 'NN'), ('com', 'NN'), ('xvzzt', 'NNP'), ('haiti', 'VBD'), ('earthquake', 'NN')]
[('info', 'NN'), ('for', 'IN'), ('the', 'DT'), ('possibility', 'NN'), ('of', 'IN'), ('employment', 'NN'), ('for', 'IN'), ('a', 'DT'), ('civil', 'JJ'), ('engineer', 'NN')]
[('today', 'NN'), ('there', 'EX'), ('is', 'VBZ'), ('a', 'DT'), ('lot', 'NN'), ('of', 'IN'), ('civil', 'JJ'), ('unrest', 'NN')]
[('we', 'PRP'), ('need', 'VBP'), ('police', 'NNS'), ('assistance', 'VB')]
[('i', 'NNS'

[('what', 'WP'), ('should', 'MD'), ('we', 'PRP'), ('do', 'VB'), ('when', 'WRB'), ('people', 'NNS'), ('are', 'VBP'), ('talking', 'VBG'), ('back', 'RB'), ('to', 'TO'), ('u', 'VB'), ('thank', 'NN'), ('you', 'PRP')]
[('according', 'VBG'), ('to', 'TO'), ('the', 'DT'), ('agreement', 'NN'), ('the', 'DT'), ('grant', 'NN'), ('will', 'MD'), ('be', 'VB'), ('used', 'VBN'), ('to', 'TO'), ('purchase', 'VB'), ('fertilizer', 'NN'), ('that', 'WDT'), ('aimed', 'VBD'), ('at', 'IN'), ('improving', 'VBG'), ('the', 'DT'), ('food', 'NN'), ('security', 'NN'), ('for', 'IN'), ('underprivileged', 'JJ'), ('farmer', 'NN'), ('in', 'IN'), ('the', 'DT'), ('country', 'NN')]
[('my', 'PRP$'), ('love', 'NN'), ('i', 'NN'), ('can', 'MD'), ('t', 'VB'), ('live', 'VB'), ('without', 'IN'), ('you', 'PRP'), ('but', 'CC'), ('ala', 'VB')]
[('tell', 'VB'), ('me', 'PRP'), ('why', 'WRB'), ('you', 'PRP'), ('are', 'VBP'), ('angry', 'JJ'), ('and', 'CC'), ('you', 'PRP'), ('are', 'VBP'), ('humiliating', 'VBG'), ('me', 'PRP')]
[('i', 'NN')

UnboundLocalError: local variable 'first_tag' referenced before assignment

In [ ]:
predictions = pipeline.predict(X_train)
eval_metrics(y_train, predictions, y.columns.values)

In [ ]:
predictions = pipeline.predict(X_test)
eval_metrics(y_test, predictions, y.columns.values)

In [ ]:
param_grid = {'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#               'features__text_pipeline__vect__max_df': (0.75, 1.0),
#               'clf__estimator__n_estimators': [100, 250, 500]
             }
model = build_model_pipeline()
scorer = make_scorer(gridsCV_scorer)
grid_model = GridSearchCV(model, param_grid=param_grid, scoring=scorer, verbose=20)
grid_model.fit(X_train, y_train)

In [ ]:
best = grid_model.best_params_
best

In [ ]:
predictions = grid_model.predict(X_test)
eval_m= eval_metrics(y_test, predictions, y.columns.values)

In [ ]:
eval_m.describe()

In [ ]:
joblib.dump(grid_model, 'DisasterResponse.pkl')

In [ ]:
# # load the model from disk
# loaded_model = load('DisasterResponse.pkl')
# result = loaded_model.score(X_test, y_test)
# print(result)

In [ ]:
grid_model